ARTIST CENTRALITY part II

Learning from Networks project 2022-2023

TODO: add info of our group

TODO: add short descritpion of the project here

Let's start by importing libraries that we'll use

In [1]:
import networkx as nx 
import time
import random as rnd
import math

Let's build our graph starting with adding the nodes

In the file *nodes.csv* datas is rappresented in the following format: ___"spotify_id,name,followers,popularity,genres,chart_hits"___.

We are interested in the spotify ID, in the name, in the number of followers and in the popularity index.

In the dataset there are some duplicates so we need to check we are not adding any by mistake and that the one we added are the one with the information we are interested (i.e. a duplicate can have the number of followers setted to 0)

N.B.: we need to be carefull splitting the line based on commas since an artist name can have a comma aswell
- without comma: 48WvrUGoijadXXCsGocwM4,Byklubben,1738.0,24,"['nordic house', 'russelater']",['no (3)']

- with comma: 7c1HgFDe8ogy5NOZ1ANCJQ,"Car, the garden",110672.0,51,"['k-indie', 'korean pop']","['id (1)', 'my (1)', 'th (1)']"

In [3]:
G = nx.Graph()

#Load dataset with nodes infos
f = open('dataset/nodes.csv', "r", encoding="utf8")

# skip the first line in the input file since it contains dataset description
f.readline()

discarded_counter=0

while True:
    line = f.readline().strip()
    
    #empty line = EOF
    if line == '':
        break
        
    #First we extrapolate the id
    current_id, tmp = line.split(',', 1)
    
    #initialize other variables
    current_artist = current_followers = current_popularity = ''
    
    #Now we divide the 2 cases, artist with comma in their name and artists without,
    #we do that by checking if the first character is equals to '"'
    if tmp[0] != '"':
        #i.e. tmp = Byklubben,1738.0,24,"['nordic house', 'russelater']",['no (3)']
        current_artist, current_followers, current_popularity, tmp = tmp.split(',', 3)
    else:
        #i.e. tmp = "Car, the garden",110672.0,51,"['k-indie', 'korean pop']","['id (1)', 'my (1)', 'th (1)']"
        empty, current_artist, tmp = tmp.split('"', 2)
        #i.e. tmp = ,110672.0,51,"['k-indie', 'korean pop']","['id (1)', 'my (1)', 'th (1)']"
        empty, current_followers, current_popularity, tmp = tmp.split(',', 3)

    #Now let's try converting followers and popularity index currently string to ints,
    #in case we can't we skip the current node reporting a message
    try:
        current_followers = int(float(current_followers))
        current_popularity = int(current_popularity)                
    except ValueError as ve:
        print('ValueError occured while converting string to int. Node ID:', current_id)
        discarded_counter += 1
        continue

    #if the ID is already a node we check we have the correct informations
    if G.has_node(current_id):
        #if the number of followers of the current line is greater than the numbers of followers already addeed
        #to the graph we just update the corresponding label
        if G.nodes[current_id]['followers'] < current_followers:
            G.nodes[current_id]['followers'] = current_followers
        #same with popularity
        if G.nodes[current_id]['popularity'] < current_popularity:
            G.nodes[current_id]['popularity'] = current_popularity
    
    #else we add it to the graph
    else:
        #add new node with mapped int as key and artist and followers as lables
        G.add_node(current_id, artist=current_artist, followers=current_followers, popularity=current_popularity, path_sum=0)


# Close opend file
f.close()
            
#print results
print(discarded_counter,"nodes have been discarded because of bad formatting!")
print(G.number_of_nodes(),"nodes have been added successfully")

ValueError occured while converting string to int. Node ID: 4Jgl9FmNQF6ontIRyY19Ig
ValueError occured while converting string to int. Node ID: 3cCFieWefBXyyDRsjNuArE
ValueError occured while converting string to int. Node ID: 1lLHQcDQFM03FcxZ5mQimA
ValueError occured while converting string to int. Node ID: 7ti7Mdu4BTfKOYWcI1Q6h8
ValueError occured while converting string to int. Node ID: 7estJE1m5cJnQs3Rc4iar0
5 nodes have been discarded because of bad formatting!
156315 nodes have been added successfully


Now let's add the edges
In the file edges.csv datas is rappresented in the following format: "id_0,id_1".
We need to check the validity of the edge before adding it because some IDs are not on the nodes.csv dataset

In [4]:
#Load dataset with edges info
f = open('dataset/edges.csv', "r", encoding="utf8")

# skip the first line in the input file since it contains dataset description
f.readline()

while True:
    line = f.readline().strip()
    
    #empty line = EOF
    if line == '':
        break
        
    #extrapolate id_0 and id_1 from the current line
    id_0, id_1 = line.split(',', 1)
    
    #if the indices are both valid then we add the edge
    if G.has_node(id_0) and G.has_node(id_1):
        G.add_edge(id_0, id_1)
        
# Close opend file
f.close()

print(G.number_of_edges(),"edges have been added successfully")

300372 edges have been added successfully


Now let's implement some approximation algorithms and run them on the full graph

**ToDo:** add some description about this algorithm

In [ ]:
def CCK_ApproximateClosenessCentrality(G,k):
    #make sure lable sum is equals to 0 for every node
    for n in G:
        G.nodes[n]['path_sum'] = 0

    sample_S0 = []
    all_Ws = []
    all_pv = []
    size_S0 = 10

    for i in range(size_S0):
        sample_S0.append(rnd.choice(list(G.nodes())))
    for s in sample_S0 :
        actualWs = 0
        sssp = nx.shortest_path_length(G, source=s)
        for n, path_lenght in sssp.items():
            G.nodes[n]['path_sum'] += path_lenght
        for n in G :
            actualWs = actualWs + G.nodes[n]['path_sum']
        all_Ws.append(actualWs)
    
    for n1 in G :
        pv = 1/len(list(G.nodes()))
        max = 0
        for s in sample_S0:
            sssp = nx.shortest_path_length(G, source=s)
            for n, path_lenght in sssp.items():
                G.nodes[n]['path_sum'] += path_lenght
            if max < G.nodes[n1]['path_sum'] :
                max = G.nodes[n1]['path_sum']
        all_pv.append(pv)

    for i in range(k):
        #pick one node based on pv (Poisson sampling)
        random_node = rnd.choices(list(G.nodes()), all_pv, 1)
        #solve sssp with picked node as source
        sssp = nx.shortest_path_length(G, source=random_node)
        #update partial sum of distancies for each node
        for n, path_lenght in sssp.items():
            G.nodes[n]['path_sum'] += path_lenght
    centralities = {}
    #compute final approximation of centrality for each node
    for n in G:
        if G.nodes[n]['path_sum'] == 0:
            centralities[n] = 0
        else:
            centralities[n] = 1/((G.number_of_nodes()*G.nodes[n]['path_sum'])/(k*(G.number_of_nodes()-1)))
    #return dicionary containg pairs (node, approximatedcentrality)
    return centralities    

**ToDo:** add a cell to call the above function

Now let's define a function for calculating the approximation using a weighted probability(instead of uniform probability) based on the degree of each node

In [ ]:
def ApproximateClosenessCentrality_NodeDegree(G, k):
    #make sure lable sum is equals to 0 for every node
    for n in G:
        G.nodes[n]['path_sum'] = 0
        
    #make a list of degree of nodes. The second elements of the list "G.degree" shows the degree of each node
    nodes_degree = [x[1] for x in G.degree]
    
    for i in range(k):
        #pick one node at random with weights based on the degree of the nodes 
        random_node = rnd.choices(list(G.nodes()), weights=list(nodes_degree), k=1)[0]

        #solve sssp with picked node as source
        sssp = nx.shortest_path_length(G, source=random_node)
        #update partial sum of distancies for each node
        for n, path_lenght in sssp.items():
            G.nodes[n]['path_sum'] += path_lenght
    centralities = {}
    #compute final approximation of centrality for each node
    for n in G:
        if G.nodes[n]['path_sum'] == 0:
            centralities[n] = 0
        else:
            centralities[n] = 1/((G.number_of_nodes()*G.nodes[n]['path_sum'])/(k*(G.number_of_nodes()-1)))
    #return dicionary containg pairs (node, approximatedcentrality)
    return centralities

Now let's run the function

In [ ]:
#let's compute the value k of the number of iteration we have to do with epsilon = 0.1
k = int(math.log(G.number_of_nodes(),2)/0.01)

start_time = time.time()
approximated_closeness_centrality_NodeDegree = ApproximateClosenessCentrality_NodeDegree(G,k)
end_time = time.time()

#let's sort the results based on the value
approximated_closeness_centrality_NodeDegree = {k: v for k, v in sorted(approximated_closeness_centrality_NodeDegree.items(),reverse=True , key=lambda item: item[1])}


#let's print the results
print("the approximated closeness centralities with %s iterations have been computed in %s seconds" %(k, end_time - start_time))

#Let's store the results
f = open('results/results_approximated_closeness_centrality_NodeDegree_epsilon_0_1.txt', "w", encoding="utf8")
for key, value in approximated_closeness_centrality_NodeDegree.items():
    f.write('%s:%s\n' %(key, value))
    
# Close opend file
f.close()

the approximated closeness centralities with 1725 iterations have been computed in 877.9626760482788 seconds


FileNotFoundError: ignored

In the next step, let's implement an approximation algorithm which runs the exact algorithm k times on different subgraphs and computes the average score. 

In [6]:
def ApproximateClosenessCentrality_KExactSubgraph(G, k):
    #here k  is the number of times we want the exact algorithm to be ran on the subgraphs
    average_closeness_centrality = []
    sumed_closeness_centrality = []
    total_closeness_centrality = []

    for i in range (k):
        #to select a subgraph at random, we should first select some nodes at random. 
        #we select subgraphs with sizes between 1000 to 2000 (at random)
        random_nodes = rnd.sample(list(G.nodes()), rnd.randint(1000, 2000))
        #generate the subgraph based on random nodes
        SG = G.subgraph(random_nodes)
        print("Number of subgraph nodes: ",SG.number_of_nodes())
        #run exact algorithm on the subgraph
        exact_closeness_centrality = nx.closeness_centrality(G)
        print("exact done")
        
        #sum the new closeness centrality values with the previous ones
        sumed_closeness_centrality = {k: exact_closeness_centrality.get(k, 0) + total_closeness_centrality.get(k, 0) for k in set(exact_closeness_centrality) | set(total_closeness_centrality)}
        #average the new closeness centrality values with the previous ones
        average_closeness_centrality = {k: (exact_closeness_centrality.get(k, 0) + total_closeness_centrality.get(k, 0))/2 for k in set(exact_closeness_centrality) & set(total_closeness_centrality)}
        
        total_closeness_centrality = sumed_closeness_centrality.update(average_closeness_centrality) 


    return total_closeness_centrality
                                  

Now let's run the function

In [ ]:
#k is the number of times to run the exact algorithm
k = 100

start_time = time.time()
approximated_closeness_centrality_KExactSubgraph = ApproximateClosenessCentrality_KExactSubgraph(G,k)
end_time = time.time()

#let's sort the results based on the value
approximated_closeness_centrality_KExactSubgraph = {k: v for k, v in sorted(approximated_closeness_centrality_KExactSubgraph.items(),reverse=True , key=lambda item: item[1])}


#let's print the results
print("the approximated closeness centralities with %s iterations have been computed in %s seconds" %(k, end_time - start_time))

#Let's store the results
f = open('results/results_approximated_closeness_centrality_KExactSubgraph.txt', "w", encoding="utf8")
for key, value in approximated_closeness_centrality_KExactSubgraph.items():
    f.write('%s:%s\n' %(key, value))
    
# Close opend file
f.close()

Number of subgraph nodes:  1040
